In [20]:
import pandas as pd
import feather
import numpy as np
import matplotlib.pyplot as plt
from scipy import optimize

Aim is to make a neural network to predict the MNIST data.
First read in data (taken from Kaggle).

In [21]:
df_raw = pd.read_csv('train.csv')

In [22]:
y = df_raw['label'].values
X = df_raw.drop(['label'], axis=1).values

Rough plan:
    
    initialise weights

    training loop:
        feedforward
        calculate loss function
        backpropogation
        update weights

    predict unlabelled data.

Will use a sigmoid activation function for simplicity.

In [23]:
def sigmoid(z):
    return 1. / (1. + np.exp(-z))

def sigmoid_gradient(z):
    g = np.zeros(z.shape)
    return sigmoid(z)*(1-sigmoid(z))

In [24]:
def cost_function(rolled_weights, input_layer_size, hidden_layer_size,
                  num_labels,X, y, lambda_=0.0):
    
    """Calculates cost function and gradients of the weight matrices.
    
    Args:
        rolled_weights (numpy array): 1D array containing the unrolled weight matrices.
        num_labels (int): The number of y data labels.
        lambda_ (float): Regularization.
        
    Returns:
        J (float): Cost function. Cross-entropy.
        grad (numpy array): 1D array containing the gradient of the unrolled weight matrices.
    """

    # Unroll rolled_weights into the weight matrices.
    Theta1 = np.reshape(rolled_weights[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(rolled_weights[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    m = y.size
    
    # Convert y to a binary matrix with each column representing a label 0-9.
    y_binary = np.zeros((y.size, num_labels))
    for i, digit in enumerate(y):
        y_binary[i, digit] = 1         

    # Feed forward
    layer1_activ = np.hstack((np.ones((X.shape[0], 1)), X))
    layer2_in = layer1_activ @ Theta1.T
    layer2_activ = np.hstack((np.ones((X.shape[0], 1)), sigmoid(layer2_in)))
    layer3_in = layer2_activ @ Theta2.T
    layer3_activ = sigmoid(layer3_in)
    delta3 = layer3_activ - y_binary
    
    # Cost function
    J = - np.sum(y_binary * np.log(layer3_activ) + (1 - y_binary) * np.log(1-layer3_activ)) / np.size(y)
    J += (lambda_ / (2*m)) * (np.sum(Theta1[:,1:]**2) + np.sum(Theta2[:,1:]**2))
    
    # Backpropogation. Slice out bias terms.
    Theta2_grad  = (1/m) * delta3.T @ layer2_activ
    Theta2_grad[:,1:] += (lambda_/m) * Theta2[:,1:]
    
    delta2 =  (Theta2.T @ delta3.T)[1:,:]
    delta2 = sigmoid_gradient(layer2_in).T *  delta2
    
    Theta1_grad = (1/m) * delta2 @ layer1_activ
    Theta1_grad[:,1:] += (lambda_/m) * Theta1[:,1:]

    # Unroll gradients
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()]) 

    return J, grad

In [25]:
def initialize_weights(nodes_in, nodes_out, epsilon=0.12):
    """Randomly nitializes the weight matrix with L_in input nodes and L_out output nodes."""
    W = np.zeros((nodes_out, 1 + nodes_in))
    W = np.random.rand(nodes_out, 1 + nodes_in) * 2 * epsilon - epsilon
    return W

In [26]:
# Training
options= {'maxiter': 300}
input_layer_size = X.shape[1]
hidden_layer_size = 25
num_labels = 10
lambda_ = 1

In [27]:
initial_Theta1 = initialize_weights(input_layer_size, hidden_layer_size)
initial_Theta2 = initialize_weights(hidden_layer_size, num_labels)
initial_rolled_weights = np.concatenate([initial_Theta1.ravel(), initial_Theta2.ravel()], axis=0)

In [28]:
# Optimize with scipy.optimize
costFunction = lambda p: cost_function(p, input_layer_size,
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)
res = optimize.minimize(costFunction, initial_rolled_weights, jac=True, 
                        method='TNC', options=options)

# Get result of optimization.
rolled_weights = res.x
Theta1 = np.reshape(rolled_weights[:hidden_layer_size * (input_layer_size + 1)],
                    (hidden_layer_size, (input_layer_size + 1)))
Theta2 = np.reshape(rolled_weights[(hidden_layer_size * (input_layer_size + 1)):],
                    (num_labels, (hidden_layer_size + 1)))

In [29]:
def predict(Theta1, Theta2, X):
    """Feedforward to predict digits."""
    m = X.shape[0]
    num_labels = Theta2.shape[0]

    p = np.zeros(m)
    h1 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), X], axis=1), Theta1.T))
    h2 = sigmoid(np.dot(np.concatenate([np.ones((m, 1)), h1], axis=1), Theta2.T))
    p = np.argmax(h2, axis=1)
    return p

In [30]:
pred = predict(Theta1, Theta2, X)
print('Training Set Accuracy: %f' % (np.mean(pred == y) * 100))

Training Set Accuracy: 86.819048


Not bad for a first attempt.

To do:
- Make a cross-validation set (or bootstrap).
- Adjust with hidden layer size, regularization and training time.
- Should be able to get up to ~95% on test data.